In [3]:
import numpy as np
import pandas as pd
import pyterrier as pt

In [2]:
if not pt.started():
    pt.init()

/tmp/ipykernel_28668/3057724015.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_28668/3057724015.py:2: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [4]:
import ir_datasets

In [5]:
dataset = ir_datasets.load('istella22/test')

In [11]:
print(dataset)
print(dataset.queries)
print(dataset.docs)
for i in dataset.docs:
    print(i)

Dataset(id='istella22/test', provides=['docs', 'queries', 'qrels'])
BetaPythonApiQueries(Dataset(id='istella22/test', provides=['docs', 'queries', 'qrels']))
BetaPythonApiDocs(Dataset(id='istella22/test', provides=['docs', 'queries', 'qrels']))


ValueError: Insufficient disk space: /home/ersel/.ir_datasets/istella22/istella22.tar.gz requires 26.5GB but only 25.6GB is available (909.6MB more needed)

In [ ]:
def dcg_at_k(relevances, k=None):
    if k:
        relevances = relevances[:k]
    return sum([rel / np.log2(i + 1) for i, rel in enumerate(relevances, 1)])

def ndcg_at_k(relevances, k=None):
    dcg = dcg_at_k(relevances, k)
    idcg = dcg_at_k(sorted(relevances, reverse=True), k)
    return dcg / idcg if idcg > 0 else 0

In [13]:
def kendalls_tau(order1, order2, n):
    concordont_count = 0
    discordont_count = 0
    for d1 in range(n):
        for d2 in range(d1+1, n):
            if ((order1[d1] > order1[d2]) == (order2[d1] > order2[d2])):
                concordont_count += 1
            else:
                discordont_count += 1
    return (concordont_count - discordont_count) / (n*(n-1)/2)

In [ ]:
class Feature:
    def __init__(self, id, score, docs):
        self.id = id
        self.score = score
        self.docs = docs

In [7]:
def get_sim_key(f1, f2):
    return str(min(f1.id, f2.id)) + "#" + str(max(f1.id, f2.id))

In [8]:
def create_feature_containers(dataset, n=1000):
    features = []
    similarity_dict = {}
    for feature in dataset.features:
        model = train_data(dataset, feature)
        relevancy_list = order_documents(model, dataset)
        score = get_score(relevancy_list)
        features.append(Feature(feature, score, relevancy_list))
    for f1_index in range(len(features)):
        f1 = features[f1_index]
        for f2_index in range(f1_index + 1, len(features)):
            f2 = features[f2_index]
            key = get_sim_key(f1, f2)
            similarity_dict[key] = kendalls_tau(f1.docs, f2.docs, min(len(f1.docs), 1000))
    return features, similarity_dict



In [27]:
def get_best_feature(features):
    best_feature = None
    for feature in features:
        if not best_feature or feature.score >= best_feature.score:
            best_feature = feature
    return best_feature

def update_features(features, best_feature, similarity_dict):
    for feature in features:
        if feature != best_feature:
            key = get_sim_key(feature, best_feature)
            sim_val = similarity_dict(key)
            feature.score -= sim_val

def GAS(features, similarity_dict):
    n = len(features)
    features_ordered = []
    for i in range(n-1):
        best_feature = get_best_feature(features)
        features_ordered.append(best_feature)
        update_features(features, best_feature, similarity_dict)
        features.remove(best_feature)
    return features_ordered

In [ ]:
"""
corpus = [
    {"docno": "1", "text": "the quick brown fox jumps over the lazy dog"},
    {"docno": "2", "text": "the lazy dog sleeps in the sun"},
    {"docno": "3", "text": "a fast fox chases the rabbit"}
]

corpus_df = pd.DataFrame(corpus)

# Index the corpus using PyTerrier's Index Factory
indexer = pt.IterDictIndexer("./index")
index_ref = indexer.index(corpus_df["text"], corpus_df["docno"])

# Load the index
index = pt.IndexFactory.of(index_ref)

# BM25 Retrieval
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

# Example query
query = "fox and dog"
query_df = pd.DataFrame([{"qid": "1", "query": query}])

# Get the results
results = bm25.transform(query_df)

# Display the results
print(results)
"""

In [ ]:

features, similarity_dict = create_feature_containers(dataset)
features = GAS(features, similarity_dict)